In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Load the data
train_data = np.load('./chunk/age/hubert_3s_train_3.npz')
test_data = np.load('./chunk/age/hubert_3s_test_3.npz')

train_features = train_data['features']  # Ensure these have 1024 features
test_features = test_data['features']    # Ensure these have 1024 features

# Read Age labels instead of Gender
train_labels = pd.read_csv('y_train.csv').Age
test_labels = pd.read_csv('y_test.csv').Age

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Handle unseen labels in test set by filtering out labels not present in train set
test_labels_filtered = test_labels[test_labels.isin(train_labels.unique())]
test_features_filtered = test_features[test_labels.isin(train_labels.unique())]
test_labels_encoded = label_encoder.transform(test_labels_filtered)

# Check for NaNs and handle them before standardization
train_features = np.nan_to_num(train_features, nan=np.nanmean(train_features))
test_features_filtered = np.nan_to_num(test_features_filtered, nan=np.nanmean(test_features_filtered))

# Standardize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features_filtered)

# SVM model
svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features_normalized, train_labels_encoded)

# SVM Prediction and Evaluation
svm_predictions = svm_model.predict(test_features_normalized)
svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# CNN model (TensorFlow)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout

train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)  # Shape: (num_samples, 1024, 1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)    # Shape: (num_samples, 1024, 1)

cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(1024, 1)),  # Updated input shape to 1024
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

cnn_model.fit(train_features_cnn, train_labels_encoded, epochs=50, batch_size=32, validation_split=0.2)

# CNN Prediction and Evaluation
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=-1)
cnn_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)
print(f"CNN Accuracy: {cnn_accuracy:.4f}")
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# ANN model (TensorFlow)
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),  # Updated input shape to 1024
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

ann_model.fit(train_features_normalized, train_labels_encoded, epochs=50, batch_size=32, validation_split=0.2)

# ANN Prediction and Evaluation
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=-1)
ann_accuracy = accuracy_score(test_labels_encoded, ann_predictions)
print(f"ANN Accuracy: {ann_accuracy:.4f}")
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
import torch
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Shape: (num_samples, 1, 1024)
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)    # Shape: (num_samples, 1, 1024)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

import torch.nn as nn
import torch.optim as optim

class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 1024, 512)  # Updated to 1024 features
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop and evaluation (not shown in this code block)


SVM Accuracy: 0.7874
SVM Classification Report:
              precision    recall  f1-score   support

           1       0.70      0.58      0.64        12
           2       0.67      0.75      0.71         8
           3       0.82      0.95      0.88        39
           4       1.00      0.83      0.91         6
           5       0.80      0.79      0.80        42
           6       0.69      0.56      0.62        16
           7       0.67      1.00      0.80         2
           8       1.00      0.50      0.67         2

    accuracy                           0.79       127
   macro avg       0.79      0.75      0.75       127
weighted avg       0.79      0.79      0.78       127



/home/mojo/.local/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 516ms/step - accuracy: 0.3695 - loss: 3.2475 - val_accuracy: 0.1919 - val_loss: 2.1069
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 520ms/step - accuracy: 0.7463 - loss: 0.7620 - val_accuracy: 0.1628 - val_loss: 2.7064
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 514ms/step - accuracy: 0.9284 - loss: 0.2281 - val_accuracy: 0.4128 - val_loss: 2.2617
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 521ms/step - accuracy: 0.9804 - loss: 0.0972 - val_accuracy: 0.3256 - val_loss: 3.1670
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 507ms/step - accuracy: 0.9912 - loss: 0.0354 - val_accuracy: 0.1919 - val_loss: 4.8830
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 519ms/step - accuracy: 0.9942 - loss: 0.0292 - val_accuracy: 0.2965 - val_loss: 3.7621
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 513ms/step - accuracy: 0.9922 - loss: 0.0213 - val_accuracy: 0.1570 - val_loss: 5.8814
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 540ms/step - accuracy: 0.9965 - loss: 0.0138 - val_accu

/home/mojo/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2535 - loss: 2.7574 - val_accuracy: 0.2384 - val_loss: 1.9334
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4317 - loss: 1.6937 - val_accuracy: 0.1977 - val_loss: 2.4188
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5143 - loss: 1.5477 - val_accuracy: 0.2267 - val_loss: 2.2287
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6115 - loss: 1.1619 - val_accuracy: 0.2326 - val_loss: 2.4571
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6811 - loss: 0.9300 - val_accuracy: 0.2965 - val_loss: 2.3994
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7162 - loss: 0.8106 - val_accuracy: 0.2791 - val_loss: 2.5659
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7546 - loss: 0.6941 - val_accuracy: 0.3314 - val_loss: 2.2672
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7635 - loss: 0.7861 - val_accuracy: 0.2733 - val_loss: 2.8120
Epo